In [12]:
import sys
import pyodbc
import db_config as cf
import pandas as pd
from IPython.display import display, HTML
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

server_name = cf.DATABASE_CONFIG['server_name']
db_name = cf.DATABASE_CONFIG['db_name']
uid = cf.DATABASE_CONFIG['user']
pwd = cf.DATABASE_CONFIG['password']
driver = '{ODBC Driver 13 for SQL Server}'
mailto = cf.MAIL_CONFIG['mail_id']
mailpw = cf.MAIL_CONFIG['mail_pw']

sql_statement = "DECLARE @sYYear char(4), @sYMonth char(2), @sYDay char(2) " \
                "DECLARE @dDate as smalldatetime, @sDate char(10), @mDate Char(10), @sDateFM char(10), @sDateTO char(10)" \
                "SET @sDateFM = CONVERT(CHAR(10), GETDATE(), 101)" \
                "SET @sDateTO = CONVERT(CHAR(10), GETDATE() + 90, 101)" \
                "SET @dDate = CONVERT(SMALLDATETIME, GETDATE() )" \
                "SET @sYYear = DATEPART(year,@dDate) SET @sYMonth = DATEPART(month,@dDate)" \
                "SET @sYDay = DATEPART(day,@dDate)" \
                "SELECT TOP 32 CONVERT(CHAR(10), CAST(YYYYMMDD AS SMALLDATETIME), 101) AS SBDATE, SHDAILY, SHACCU, BKDAILY, BKACCU, DAYOFWEEK, ISNULL(SHdailyUSNPI, 0) AS SHdailyUSNPI, ISNULL(SHAccuUSNPI, 0) AS SHAccuUSNPI, ISNULL(SHdailyUSEMS, 0) AS SHdailyUSEMS, ISNULL(SHAccuUSEMS, 0) AS SHAccuUSEMS, ISNULL(SHdailyKR, 0) AS SHdailyKR, ISNULL(SHAccuKR, 0) AS SHAccuKR, ISNULL(SHdailyChina, 0) AS SHdailyChina, ISNULL(SHAccuChina, 0) AS SHAccuChina,  ISNULL(SHDailyOther, 0) AS SHDailyOther, ISNULL(SHAccuOther, 0) AS SHAccuOther, ISNULL(SHDailyIntSales, 0) AS SHDailyIntSales, ISNULL(SHAccuIntSales, 0) AS SHAccuIntSales, ISNULL(BKDailyUS, 0) AS BKDailyUS, ISNULL(BKAccuUS, 0) AS BKAccuUS, ISNULL(BKDailyKR, 0) AS BKDailyKR, ISNULL(BKAccuKR, 0) AS BKAccuKR, ISNULL(PASTDUE, 0) AS PASTDUE, ISNULL(NPIPAST,   0) AS NPIPAST, ISNULL(RMAPAST, 0) AS RMAPAST, ISNULL(BOOKWITHIN90, 0)  AS BOOK90, ISNULL(TOTALBOOK, 0) AS TOTALBOOK  FROM  TE6050_HISTORY WHERE  LEFT(YYYYMMDD, 6) =   @sYYear + @sYMonth ORDER  BY SBDATE DESC"


cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server_name+';PORT=1433;DATABASE='+db_name+';UID='+uid+';PWD='+ pwd)

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '$ {:,.2f}'.format)
df = pd.read_sql(sql_statement, cnxn)
list(df)

['SBDATE',
 'SHDAILY',
 'SHACCU',
 'BKDAILY',
 'BKACCU',
 'DAYOFWEEK',
 'SHdailyUSNPI',
 'SHAccuUSNPI',
 'SHdailyUSEMS',
 'SHAccuUSEMS',
 'SHdailyKR',
 'SHAccuKR',
 'SHdailyChina',
 'SHAccuChina',
 'SHDailyOther',
 'SHAccuOther',
 'SHDailyIntSales',
 'SHAccuIntSales',
 'BKDailyUS',
 'BKAccuUS',
 'BKDailyKR',
 'BKAccuKR',
 'PASTDUE',
 'NPIPAST',
 'RMAPAST',
 'BOOK90',
 'TOTALBOOK']

In [17]:
df = df.set_index('SBDATE')
df.head()

,SHDAILY,SHACCU,BKDAILY,BKACCU,DAYOFWEEK,SHdailyUSNPI,SHAccuUSNPI,SHdailyUSEMS,SHAccuUSEMS,SHdailyKR,SHAccuKR,SHdailyChina,SHAccuChina,SHDailyOther,SHAccuOther,SHDailyIntSales,SHAccuIntSales,BKDailyUS,BKAccuUS,BKDailyKR,BKAccuKR,PASTDUE,NPIPAST,RMAPAST,BOOK90,TOTALBOOK
SBDATE,,,,,,,,,,,,,,,,,,,,,,,,,,
10/04/2018,"$ 26,174.43","$ 138,759.90",$ 0.00,"$ 425,422.89",Thursday,$ 0.00,"$ 3,532.48","$ 2,168.07","$ 58,405.27","$ 24,006.36","$ 76,822.15",$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,"$ 392,377.89",$ 0.00,"$ 33,045.00","$ 11,473.82",$ 0.00,$ 0.00,"$ 7,008,605.99","$ 10,535,625.43"
10/03/2018,"$ 59,151.22","$ 112,585.47","$ 81,622.18","$ 425,422.89",Wednesday,$ 0.00,"$ 3,532.48","$ 13,252.47","$ 56,237.20","$ 45,898.75","$ 52,815.79",$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,"$ 48,577.18","$ 392,377.89","$ 33,045.00","$ 33,045.00",$ 372.32,$ 0.00,$ 0.00,"$ 6,861,551.62","$ 10,561,799.86"
10/02/2018,"$ 21,240.48","$ 53,434.25","$ 352,283.30","$ 343,800.71",Tuesday,$ 500.00,"$ 3,532.48","$ 20,740.48","$ 42,984.73",$ 0.00,"$ 6,917.04",$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,"$ 352,283.30","$ 343,800.71",$ 0.00,$ 0.00,"$ 32,866.93",$ 0.00,$ 0.00,"$ 6,730,202.65","$ 10,539,328.90"
10/01/2018,"$ 32,193.77","$ 32,193.77","$ -8,482.59","$ -8,482.59",Monday,"$ 3,032.48","$ 3,032.48","$ 22,244.25","$ 22,244.25","$ 6,917.04","$ 6,917.04",$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,"$ -8,482.59","$ -8,482.59",$ 0.00,$ 0.00,"$ 21,122.57",$ 0.00,$ 0.00,"$ 6,534,796.31","$ 10,208,286.08"
